# Half-Cat Rocket Motor Simulator

This notebook runs a rocket engine simulation for a blowdown feed system and uses a genetic algorithm to optimize the design for maximum total impulse.

**Instructions:**
1. This notebook is designed to be run directly from a GitHub repository using the 'Open in Colab' button.
2. The first code cell will clone the repository to make the necessary CSV files available.
3. Run the cells sequentially to perform the simulation and optimization.

In [ ]:
import pandas as pd
import math
import csv
import string
import numpy as np
from scipy.stats import linregress
from collections import deque
import random
import os

## 1. Clone Repository and Load Data

Run the following cell to clone the GitHub repository containing the required CSV files.

In [ ]:
!git clone https://github.com/<YOUR_GITHUB_USERNAME>/<YOUR_REPOSITORY_NAME>.git
os.chdir('/content/<YOUR_REPOSITORY_NAME>')

# Define the paths to your CSV files
N2O_PATH = "N2O.csv"
FUEL_PATH = "Fuel.csv"
FORMULAS_PATH = "formulas.csv"
CALCULATIONS_PATH = "Calculations.csv"
ISENTROPIC_FLOW_PATH = "Isentropic Flow.csv"

# Load data once
n2o_data = load_csv(N2O_PATH)
fuel_data = load_csv(FUEL_PATH)
formulas_data = load_csv(FORMULAS_PATH)
calculations_data = load_csv(CALCULATIONS_PATH)
isentropic_flow_data = load_csv(ISENTROPIC_FLOW_PATH)

## 2. Function Definitions

This cell contains all the necessary functions from the `halfcatsim.py` script.

In [ ]:
def INTERCEPT(
    dataframe: pd.DataFrame,
    y_column_name: str,
    x_column_name: str,
    start_row: int,
    end_row: int
  ) -> float:
    if y_column_name not in dataframe.columns:
        raise ValueError(f"Y-column '{y_column_name}' not found in the DataFrame.")
    if x_column_name not in dataframe.columns:
        raise ValueError(f"X-column '{x_column_name}' not found in the DataFrame.")

    num_rows = len(dataframe)
    if not (0 <= start_row <= end_row < num_rows):
        raise ValueError(f"Invalid row range: start_row={start_row}, end_row={end_row}. "
                         f"DataFrame has {num_rows} rows (0-indexed up to {num_rows - 1}).")

    selected_df_range = dataframe.iloc[start_row : end_row + 1]

    y_series = selected_df_range[y_column_name]
    x_series = selected_df_range[x_column_name]

    combined_data = pd.DataFrame({
        'x': x_series,
        'y': y_series
    }).dropna()

    if combined_data.empty:
        raise ValueError("No valid (non-NaN) data points found in the specified columns and row range for regression.")

    try:
        x_values = pd.to_numeric(combined_data['x'], errors='coerce').dropna().values
        y_values = pd.to_numeric(combined_data['y'], errors='coerce').dropna().values
    except Exception as e:
        raise ValueError(f"Could not convert data in specified columns to numeric: {e}")

    if len(x_values) != len(y_values):
        raise ValueError("After numeric conversion and NaN removal, X and Y data points do not align for regression.")

    if len(x_values) < 2:
        raise ValueError("Not enough valid numeric data points (need at least 2) for linear regression in the specified range.")

    slope, intercept, r_value, p_value, std_err = linregress(x_values, y_values)

    return intercept

def _col_to_index(col):
    if isinstance(col, int):
        if col < 1:
            raise ValueError(f"Column number must be 1 or greater, got {col}.")
        return col - 1
    elif isinstance(col, str):
        col = col.upper()
        index = 0
        for char in col:
            if not 'A' <= char <= 'Z':
                raise ValueError(f"Invalid column letter: '{col}'. Must be A-Z.")
            index = index * 26 + (ord(char) - ord('A') + 1)
        return index - 1
    else:
        raise TypeError(f"Column must be an integer or a string letter, got {type(col)}.")

def slice_data(full_table_data, start_row, end_row, start_col, end_col):
    if not isinstance(full_table_data, list) or not full_table_data:
        raise ValueError("full_table_data must be a non-empty list of lists.")
    if not all(isinstance(row, list) for row in full_table_data):
        raise ValueError("full_table_data must contain only lists (rows).")

    if not (isinstance(start_row, int) and start_row >= 1 and
            isinstance(end_row, int) and end_row >= start_row):
        raise ValueError("start_row and end_row must be integers, 1 or greater, and start_row <= end_row.")

    python_start_row = start_row - 1
    python_end_row = end_row

    python_start_col = _col_to_index(start_col)
    python_end_col = _col_to_index(end_col) + 1

    if python_start_col >= python_end_col:
        raise ValueError(f"start_col ('{start_col}') must be less than end_col ('{end_col}'). This indicates a backward range in Excel, which needs to be reordered (min to max column).")


    sliced_data = []
    for r_idx in range(python_start_row, min(python_end_row, len(full_table_data))):
        row = full_table_data[r_idx]
        if not row:
            sliced_data.append([])
            continue

        if len(row) > python_start_col:
            sliced_row = row[python_start_col:min(python_end_col, len(row))]
            sliced_data.append(sliced_row)
        else:
            sliced_data.append([])

    return sliced_data

def load_csv(file_path):
    table_data = []
    try:
        with open(file_path, 'r', newline='') as file:
            csv_reader = csv.reader(file)
            for row in csv_reader:
                processed_row = []
                for i, item in enumerate(row):
                    if i == 0:
                        try:
                            processed_row.append(float(item))
                        except ValueError:
                            processed_row.append(item)
                    else:
                        processed_row.append(item)
                table_data.append(processed_row)
        return table_data
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return None
    except Exception as e:
        print(f"An error occurred while reading '{file_path}': {e}")
        return None

_CONVERSION_MAP = {
    "length": {
        "m": 1.0, "meter": 1.0, "meters": 1.0, "km": 1000.0, "kilometer": 1000.0, "kilometers": 1000.0,
        "cm": 0.01, "centimeter": 0.01, "centimeters": 0.01, "mm": 0.001, "millimeter": 0.001, "millimeters": 0.001,
        "in": 0.0254, "inch": 0.0254, "inches": 0.0254, "ft": 0.3048, "foot": 0.3048, "feet": 0.3048,
        "yd": 0.9144, "yard": 0.9144, "yards": 0.9144, "mi": 1609.344, "mile": 1609.344, "miles": 1609.344,
    },
    "mass": {
        "kg": 1.0, "kilogram": 1.0, "kilograms": 1.0, "g": 0.001, "gram": 0.001, "grams": 0.001,
        "lbm": 0.45359237, "lb": 0.45359237, "pound": 0.45359237, "pounds": 0.45359237,
        "ozm": 0.028349523125, "oz": 0.028349523125, "ounce": 0.028349523125, "ounces": 0.028349523125,
    },
    "time": {
        "s": 1.0, "sec": 1.0, "second": 1.0, "seconds": 1.0, "min": 60.0, "minute": 60.0, "minutes": 60.0,
        "hr": 3600.0, "hour": 3600.0, "hours": 3600.0, "day": 86400.0, "days": 86400.0,
    },
    "volume": {
        "l": 1.0, "liter": 1.0, "liters": 1.0, "ml": 0.001, "milliliter": 0.001, "milliliters": 0.001,
        "gal": 3.785411784, "gallon": 3.785411784, "gallons": 3.785411784, "qt": 0.946352946,
        "quart": 0.946352946, "quarts": 0.946352946, "pt": 0.473176473, "pint": 0.473176473, "pints": 0.473176473,
        "cup": 0.2365882365, "cups": 0.2365882365, "foz": 0.0295735295625, "fluid_ounce": 0.0295735295625, "fluid_ounces": 0.0295735295625,
        "m3": 1000.0, "cubic_meter": 1000.0, "cubic_meters": 1000.0, "ft3": 28.316846592, "cubic_foot": 28.316846592,
        "cubic_feet": 28.316846592, "in3": 0.016387064, "cubic_inch": 0.016387064, "cubic_inches": 0.016387064,
    },
    "temperature": {
        "c": "celsius", "celsius": "celsius", "f": "fahrenheit", "fahrenheit": "fahrenheit",
        "k": "kelvin", "kelvin": "kelvin",
    },
    "pressure": {
        "pa": 1.0, "pascal": 1.0, "pascals": 1.0,
        "kpa": 1000.0, "kilopascal": 1000.0, "kilopascals": 1000.0,
        "mpa": 1000000.0, "megapascal": 1000000.0, "megapascals": 1000000.0,
        "psi": 6894.757293168, "pound_per_square_inch": 6894.757293168,
        "bar": 100000.0,
        "atm": 101325.0, "atmosphere": 101325.0, "atmospheres": 101325.0,
        "mmhg": 133.322387415, "millimeter_of_mercury": 133.322387415, "torr": 133.322387415,
        "inhg": 3386.388666667, "inch_of_mercury": 3386.388666667,
    },
    "area": {
        "m2": 1.0, "m^2": 1.0, "sqm": 1.0, "square_meter": 1.0, "square_meters": 1.0,
        "cm2": 0.0001, "cm^2": 0.0001, "sqcm": 0.0001, "square_centimeter": 0.0001, "square_centimeters": 0.0001,
        "mm2": 0.000001, "mm^2": 0.000001, "sqmm": 0.000001, "square_millimeter": 0.000001, "square_millimeters": 0.000001,
        "in2": 0.00064516, "in^2": 0.00064516, "sqin": 0.00064516, "square_inch": 0.00064516, "square_inches": 0.00064516,
        "ft2": 0.09290304, "ft^2": 0.09290304, "sqft": 0.09290304, "square_foot": 0.09290304, "square_feet": 0.09290304,
        "yd2": 0.83612736, "yd^2": 0.83612736, "sqyd": 0.83612736, "square_yard": 0.83612736, "square_yards": 0.83612736,
        "mi2": 2589988.110336, "mi^2": 2589988.110336, "sqmi": 2589988.110336, "square_mile": 2589988.110336, "square_miles": 2589988.110336,
        "acre": 4046.8564224, "acres": 4046.8564224,
        "ha": 10000.0, "hectare": 10000.0, "hectares": 10000.0,
    },
    "force": {
        "n": 1.0, "newton": 1.0, "newtons": 1.0,
        "kn": 1000.0, "kilonewton": 1000.0, "kilonewtons": 1000.0,
        "lbf": 4.44822, "pound_force": 4.44822, "pound-force": 4.44822, "pounds_force": 4.44822, "pounds-force": 4.44822,
        "kgf": 9.80665, "kilogram_force": 9.80665, "kilogram-force": 9.80665, "kilopond": 9.80665, "kp": 9.80665,
    }
}

_UNIT_TO_CATEGORY = {unit: category for category, units in _CONVERSION_MAP.items() for unit in units}

def _get_unit_category(unit_str):
    normalized_unit = unit_str.lower()
    return _UNIT_TO_CATEGORY.get(normalized_unit)

def _convert_temperature(value, from_unit, to_unit):
    normalized_from = from_unit.lower()
    normalized_to = to_unit.lower()

    if normalized_from == "c" or normalized_from == "celsius":
        celsius_value = value
    elif normalized_from == "f" or normalized_from == "fahrenheit":
        celsius_value = (value - 32) * 5 / 9
    elif normalized_from == "k" or normalized_from == "kelvin":
        celsius_value = value - 273.15
    else:
        raise ValueError(f"Unknown temperature 'from' unit: {from_unit}")

    if normalized_to == "c" or normalized_to == "celsius":
        return celsius_value
    elif normalized_to == "f" or normalized_to == "fahrenheit":
        return (celsius_value * 9 / 5) + 32
    elif normalized_to == "k" or normalized_to == "kelvin":
        return celsius_value + 273.15
    else:
        raise ValueError(f"Unknown temperature 'to' unit: {to_unit}")


def CONVERT(number, from_unit, to_unit):
    if not isinstance(number, (int, float)):
        raise ValueError("CONVERT function: 'number' must be a numeric value.")

    from_unit_lower = from_unit.lower()
    to_unit_lower = to_unit.lower()

    from_category = _get_unit_category(from_unit_lower)
    to_category = _get_unit_category(to_unit_lower)

    if from_category is None:
        raise ValueError(f"CONVERT function: Unrecognized 'from_unit': '{from_unit}'")
    if to_category is None:
        raise ValueError(f"CONVERT function: Unrecognized 'to_unit': '{to_unit}'")

    if from_category != to_category:
        raise ValueError(f"CONVERT function: Cannot convert between incompatible unit categories ('{from_category}' and '{to_category}').")

    if from_category == "temperature":
        return _convert_temperature(number, from_unit_lower, to_unit_lower)
    else:
        try:
            from_factor = _CONVERSION_MAP[from_category][from_unit_lower]
            base_value = number * from_factor
            to_factor = _CONVERSION_MAP[to_category][to_unit_lower]
            converted_value = base_value / to_factor
            return converted_value
        except KeyError:
            raise ValueError("Internal error: Unit not found in conversion map despite category being identified.")
        except Exception as e:
            raise ValueError(f"An unexpected error occurred during conversion: {e}")

def VLOOKUP(lookup_value, table_array, col_index_num, range_lookup=True):
    if not isinstance(table_array, (list, tuple)) or not table_array:
        raise ValueError("VLOOKUP: 'table_array' must be a non-empty list or tuple of rows.")

    if not isinstance(col_index_num, int) or col_index_num < 1:
        raise ValueError("VLOOKUP: 'col_index_num' must be an integer greater than or equal to 1.")

    target_col_index = col_index_num - 1

    if not table_array[0]:
        raise ValueError("VLOOKUP: 'table_array' contains empty rows.")
    if target_col_index >= len(table_array[0]):
        raise ValueError(f"VLOOKUP: 'col_index_num' ({col_index_num}) is out of bounds. "
                         f"Table has {len(table_array[0])} columns.")

    if range_lookup is False:
        for row in table_array:
            if not row: continue
            try:
                if isinstance(lookup_value, (int, float)):
                    first_col_value = type(lookup_value)(row[0])
                else:
                    first_col_value = row[0]
            except ValueError:
                continue

            if first_col_value == lookup_value:
                return row[target_col_index]
        return None
    else:
        last_suitable_match = None
        for row in table_array:
            if not row: continue
            try:
                first_col_value = type(lookup_value)(row[0])
            except ValueError:
                continue

            if first_col_value <= lookup_value:
                last_suitable_match = row[target_col_index]
            else:
                break
        return last_suitable_match

def MATCH(lookup_value, lookup_array, match_type=0):
    if match_type == 0:
        for i, val in enumerate(lookup_array):
            try:
                if isinstance(lookup_value, (int, float)):
                    val_converted = float(val)
                else:
                    val_converted = val
            except ValueError:
                if isinstance(lookup_value, (int, float)):
                    continue
                else:
                    val_converted = val
            if val_converted == lookup_value:
                return i + 1
        return None
    elif match_type == 1:
        last_suitable_match_index = None
        for i, val in enumerate(lookup_array):
            try:
                val_converted = float(val)
            except ValueError:
                continue
            if val_converted <= lookup_value:
                last_suitable_match_index = i + 1
            else:
                break
        return last_suitable_match_index
    elif match_type == -1:
        last_suitable_match_index = None
        for i, val in enumerate(lookup_array):
            try:
                val_converted = float(val)
            except ValueError:
                continue
            if val_converted >= lookup_value:
                last_suitable_match_index = i + 1
            else:
                break
        return last_suitable_match_index
    else:
        raise ValueError("MATCH: 'match_type' must be 0 for exact match, 1 for less than or exact match, or -1 for greater than or exact match.")

def SIGN(x):
    if x > 0: return 1
    if x < 0: return -1
    return 0

def calculate_result(dataframe: pd.DataFrame, column_name: str, start_row: int, end_row: int) -> float:
    if column_name not in dataframe.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

    if not (0 <= start_row <= end_row < len(dataframe)):
        raise ValueError(f"Invalid row range: start_row={start_row}, end_row={end_row}. "
                         f"DataFrame has {len(dataframe)} rows.")

    # Select the relevant slice of the Series and calculate the mean
    selected_values = dataframe.loc[start_row : end_row, column_name]

    # Check if the selected_values Series is empty (e.g., if the range was valid but no numeric data)
    if selected_values.empty:
        return 0.0 # Or raise an error, depending on desired behavior for empty ranges

    return selected_values.mean()

def get_fuel_property(pc_psi, c_star_eff, mr, table, match_array):
    """Helper to look up fuel properties."""
    if mr is None or pc_psi is None: return np.nan
    lookup_val = 50 if round(pc_psi / c_star_eff / 50, 0) * 50 == 0 else round(pc_psi / c_star_eff / 50, 0) * 50
    # Round MR to nearest 0.25
    mr_lookup = round(mr / 0.25, 0) * 0.25
    col_idx = MATCH(mr_lookup, match_array, 0)
    if col_idx is None:
        # This can happen if MR is out of range of the table
        # Returning NaN to indicate failure, which will propagate.
        return np.nan
    vlookup_result = VLOOKUP(lookup_val, table, col_idx, True)
    return float(vlookup_result) if vlookup_result is not None else np.nan

def run_simulation(
    n2o_data,
    fuel_data,
    formulas_data,
    calculations_data,
    isentropic_flow_data,
    timestep: float = 0.01,
    altitude: float = 0,
    altitude_units: str = "ft",
    throat_diameter: float = 1,
    throat_diameter_units: str = "in",
    exit_diameter: float = 2,
    exit_diameter_units: str = "in",
    exit_half_angle: float = 15,
    nozzle_efficiency: float = 0.98,
    chamber_diameter: float = 2,
    chamber_diameter_units: str = "in",
    chamber_length: float = 5,
    chamber_length_units: str = "in",
    c_star_efficiency: float = 0.70,
    ox_tank_outer_diameter: float = 4,
    ox_tank_outer_diameter_units: str = "in",
    ox_wall_thickness: float = 0.125,
    ox_wall_thickness_units: str = "in",
    siphon_diameter: float = 0,
    siphon_diameter_units: str = "in",
    ox_displacement: float = 24,
    ox_displacement_units: str = "in",
    n2o_temperature: float = 50,
    n2o_temperature_units: str = "F",
    decay_constant_liquid: float = 0.7,
    decay_constant_gas: float = 0.25,
    fuel: str = "E85",
    tank_style: str = "Stacked",
    f_tank_outer_diameter: float = 4,
    f_tank_outer_diameter_units: str = "in",
    f_wall_thickness: float = 0.125,
    f_wall_thickness_units: str = "in",
    f_tank_length: float = 12,
    f_tank_length_units: str = "in",
    f_displacement: float = 12,
    f_displacement_units: str = "in",
    piston_loss: float = 15,
    piston_loss_units: str = "psi",
    ox_feed_line_diameter: float = 0.3125,
    ox_feed_line_diameter_units: str = "in",
    ox_line_roughness: float = 3.94e-06,
    ox_line_roughness_units: str = "in",
    ox_line_length: float = 12,
    ox_line_length_units: str = "in",
    ox_component_Cv: float = 1.6,
    ox_additional_loss: float = 0,
    f_feed_line_diameter: float = 0.2250,
    f_feed_line_diameter_units: str = "in",
    f_line_roughness: float = 3.94e-06,
    f_line_roughness_units: str = "in",
    f_line_length: float = 48,
    f_line_length_units: str = "in",
    f_component_Cv: float = 1.6,
    f_additional_loss: float = 0,
    regen_channel_loss: float = 0,
    ox_injector_hole_diameter: float = 0.089,
    ox_number_of_holes: int = 6,
    ox_discharge_coefficient: float = 0.4,
    f_injector_hole_diameter: float = 0.0469,
    f_number_of_holes: int = 6,
    f_discharge_coefficient: float = 0.55
):
    """
    Runs a rocket engine simulation for a blowdown feed system.
    """

    # pre calculation outputs
    max_sim_time = timestep * 2500
    ambient_pressure_units = "psi"
    ambient_pressure = CONVERT(101325 * (1 - 0.0000225577 * CONVERT(altitude, altitude_units, "m"))**5.25588, "Pa", ambient_pressure_units)
    throat_area = CONVERT(throat_diameter, throat_diameter_units, "m")**2 * math.pi / 4
    exit_area = CONVERT(exit_diameter, exit_diameter_units, "m")**2 * math.pi / 4
    expansion_ratio = exit_area / throat_area
    contraction_ratio = chamber_diameter**2 / throat_diameter**2
    chamber_volume = math.pi * CONVERT(chamber_diameter, chamber_diameter_units, "m")**2 / 4 * CONVERT(chamber_length, chamber_length_units, "m")
    L_star_units = "in"
    L_star = CONVERT(chamber_volume / throat_area, "m", L_star_units)
    ox_tank_inner_diameter = ox_tank_outer_diameter - 2 * ox_wall_thickness
    n2o_density = float(VLOOKUP(n2o_temperature, n2o_data, 3, True))
    f_tank_inner_diameter = f_tank_outer_diameter - 2 * f_wall_thickness
    fuel_density = float(VLOOKUP(fuel, slice_data(fuel_data,4,38,"E","G"), 3, False))
    oxidizer_volume = math.pi * ox_tank_inner_diameter**2 * ox_displacement / 4 - math.pi * f_tank_outer_diameter**2 * f_tank_length / 4 - math.pi * siphon_diameter**2 * (ox_displacement - f_tank_length) / 4 if tank_style == "Concentric" else math.pi * ox_tank_inner_diameter**2 * ox_displacement / 4
    if tank_style == "Concentric":
      if siphon_diameter == 0:
        if f_tank_length < (ox_displacement / 2):
          useable_volume = oxidizer_volume - math.pi * f_tank_outer_diameter**2 * f_displacement / 4
        else:
          useable_volume = math.pi * (ox_tank_inner_diameter**2 - f_tank_outer_diameter**2) * ox_displacement / 4
      else:
        useable_volume = oxidizer_volume
    else:
      useable_volume = oxidizer_volume
    total_oxidizer_mass_units = "lbm"
    total_oxidizer_mass = CONVERT(useable_volume * n2o_density / 61024, "kg", total_oxidizer_mass_units)
    oxidizer_mass_units = "lbm"
    oxidizer_mass = CONVERT(oxidizer_volume * n2o_density / 61024, "kg", oxidizer_mass_units)
    f_tank_volume = math.pi * f_tank_outer_diameter**2 * f_tank_length / 4
    fuel_volume = math.pi * f_tank_inner_diameter**2 * f_displacement / 4
    fuel_mass_units = "lbm"
    fuel_mass = CONVERT(fuel_volume * fuel_density / 61024, "kg", fuel_mass_units)
    ox_feed_line_area = CONVERT(math.pi * ox_feed_line_diameter**2 / 4, "in^2", "m^2")
    f_feed_line_area = CONVERT(math.pi * f_feed_line_diameter**2 / 4, "in^2", "m^2")
    oxidizer_CdA = CONVERT(math.pi * ox_injector_hole_diameter**2 / 4 * ox_discharge_coefficient * ox_number_of_holes, "in^2", "m^2")
    fuel_CdA = CONVERT(math.pi * f_injector_hole_diameter**2 / 4 * f_discharge_coefficient * f_number_of_holes, "in^2", "m^2")

    # Optimization: Pre-calculate lookup tables from fuel_data
    prop_indices_table = slice_data(fuel_data, 4, 7, 'B', 'C')
    fuel_indices_table = slice_data(fuel_data, 4, 38, 'E', 'F')

    try:
        gam0_start_row = int(VLOOKUP("gam0", prop_indices_table, 2, False))
        mw_start_row = int(VLOOKUP("MW", prop_indices_table, 2, False))
        cstar_start_row = int(VLOOKUP("c*", prop_indices_table, 2, False))
        t0_start_row = int(VLOOKUP("T0", prop_indices_table, 2, False))
        fuel_start_col = int(VLOOKUP(fuel, fuel_indices_table, 2, False))
    except (TypeError, ValueError):
        print(f"Error: Could not find required properties for fuel '{fuel}' in fuel data. Check fuel name and data structure.")
        return {}

    def get_fuel_property_table(start_row, start_col):
        return slice_data(fuel_data, start_row, start_row + 20, start_col, start_col + 40)

    gam0_table = get_fuel_property_table(gam0_start_row, fuel_start_col)
    mw_table = get_fuel_property_table(mw_start_row, fuel_start_col)
    cstar_table = get_fuel_property_table(cstar_start_row, fuel_start_col)
    t0_table = get_fuel_property_table(t0_start_row, fuel_start_col)

    gam0_match_array = gam0_table[0]
    mw_match_array = mw_table[0]
    cstar_match_array = cstar_table[0]
    t0_match_array = t0_table[0]

    # Initialize the 'data' dictionary
    data = {}

    if formulas_data is None or calculations_data is None:
        print("Could not process due to file loading errors.")
        return {}
    else:
        if len(formulas_data) < 1 or len(calculations_data) < 2:
            print("Error: 'formulas.csv' needs at least one row (header) and 'Calculations.csv' needs at least two rows (header + data row).")
            return {}
        else:
            calculations_headers = calculations_data[0]
            formulas_row1 = formulas_data[0]
            calculations_row2 = calculations_data[1]

            for col_idx, header in enumerate(calculations_headers):
                key = header
                if col_idx < len(formulas_row1) and col_idx < len(calculations_row2):
                    if formulas_row1[col_idx] == 'None':
                        try:
                            data[key] = float(calculations_row2[col_idx])
                        except ValueError:
                            data[key] = calculations_row2[col_idx]
                    else:
                        data[key] = None
                else:
                    print(f"Warning: Column index {col_idx} (header '{header}') is out of bounds for row 1 in formulas.csv or row 2 in calculations.csv. Skipping.")

    # Optimization: use deque for history and pre-allocated lists for results
    history = deque(maxlen=4)
    num_steps = 2500
    results = {key: [None] * num_steps for key in data.keys()}

    init_P_N2O_tank = 0 # Initialize here
    init_m_ox = 0 # Initialize here
    for i in range(num_steps):
      if i > 0:
        data['Time'] = None if data['F'] == 0 else data['Time'] + data['Step']
      if i == 15:
        data['Step'] = 0.01
      if i == 0:
        data['P_N2O_tank'] = float(VLOOKUP(n2o_temperature, n2o_data, 2, True))
      elif i < 4:
        if data['N2O Status'] == "Liquid":
          if data['m_ox'] > 0 and data['m_f'] > 0:
            data['P_N2O_tank'] = init_P_N2O_tank * (data['m_ox'] / init_m_ox * (1 - decay_constant_liquid) + decay_constant_liquid)
          else:
            data['P_N2O_tank'] = None
            break
        else:
          data['P_N2O_tank'] = data['P_N2O_tank'] / math.exp(decay_constant_gas * (data['Time'] - (data['Time'] - data['Step'])))
      else:
        if data['N2O Status'] == "Liquid":
          if data['m_ox'] > 0 and data['m_f'] > 0:
            data['P_N2O_tank'] = init_P_N2O_tank * (data['m_ox'] / init_m_ox * (1 - decay_constant_liquid) + decay_constant_liquid)
          else:
            data['P_N2O_tank'] = None
            break
        else:
          data['P_N2O_tank'] = data['P_N2O_tank'] / math.exp(decay_constant_gas * (data['Time'] - (data['Time'] - data['Step'])))
      data['P_N2O_tank_psi'] = CONVERT(data['P_N2O_tank'] * 1000, "Pa", "psi")
      if i < 2:
        data['P_N2O_inj'] = data['P_N2O_tank']
      else:
        data['P_N2O_inj'] = data['P_N2O_tank'] - data['dP_o_total'] / 1000
      data['P_fu_tank'] = data['P_N2O_tank'] - CONVERT(piston_loss, "psi", "Pa") / 1000
      data['P_fu_tank_psi'] = CONVERT(data['P_fu_tank'] * 1000, "Pa", "psi")
      if i < 2:
        data['P_fu_inj'] = data['P_fu_tank']
      else:
        data['P_fu_inj'] = data['P_fu_tank'] - data['dP_f_total'] / 1000
      if i >= 2:
        data["gam0"] = get_fuel_property(data['Pc_psi'], c_star_efficiency, data['MR'], gam0_table, gam0_match_array)
        data["MW"] = get_fuel_property(data['Pc_psi'], c_star_efficiency, data['MR'], mw_table, mw_match_array)
      data['R'] = 8314 / data['MW']
      if i == 1:
        data['c*'] = 1000
      elif i >= 2 and i < 15:
        data["c*"] = get_fuel_property(data['Pc_psi'], c_star_efficiency, data['MR'], cstar_table, cstar_match_array)
      elif i >= 15:
        c_star_val = get_fuel_property(data['Pc_psi'], c_star_efficiency, data['MR'], cstar_table, cstar_match_array)
        data["c*"] = (c_star_val + data['c*']) / 2
      if i == 1:
        data['╢D1'] = 0.75
      elif i >= 2:
        if data['N2O Status'] == "Gas" and history[1]['N2O Status'] == "Liquid":
          data['╢D1'] = 1
        else:
          if data['Pc'] < history[1]['Pc']:
            data['╢D1'] = data['Pc'] / history[1]['Pc']
          else:
            data['╢D1'] = 1
      if i > 0:
        data['Pc_raw'] = c_star_efficiency * data['c*'] * data['m_dot'] / throat_area / 1000
        data['dP/dt'] = (data['Pc_raw'] - data['Pc']) / data['Step']
      if i < 10 and i > 0:
        if data['m_ox'] > 0 and data['m_f'] > 0:
          if data['N2O Status'] == "Gas" and history[1]['N2O Status'] == "Liquid":
            data['Pc'] = (data['Pc'] / 2) * data['╢D1'] * (0.1 * i)
          else:
            data['Pc'] = data['Pc_raw'] * data['╢D1'] * (0.1 * i)
        else:
          data['Pc'] = None
          break
      elif i >= 10 and i < 1 and i < 15:
        sum = 0
        if data['m_ox'] > 0 and data['m_f'] > 0:
          if data['N2O Status'] == "Gas" and history[1]['N2O Status'] == "Liquid":
            sum = (data['Pc_raw'] + history[1]['Pc']) / 2 * data['╢D1']
          else:
            sum = data['Pc_raw'] * data['╢D1']
          data['Pc'] = (sum + data['Pc']) / 2
        else:
          data['Pc'] = None
          break
      elif i >= 15:
        sum = 0
        if data['m_ox'] > 0 and data['m_f'] > 0:
          if data['N2O Status'] == "Gas" and history[1]['N2O Status'] == "Liquid":
            sum = (data['Pc_raw'] + data['Pc']) / 2
          else:
            if SIGN(data['dP/dt']) != SIGN(history[0]['dP/dt']) and SIGN(data['dP/dt']) == SIGN(history[1]['dP/dt']) and SIGN(data['dP/dt']) != SIGN(history[2]['dF/dt']):
              sum = (data['Pc_raw'] + data['Pc']) / 2
            else:
              sum = data['Pc_raw']
        else:
          data['F'] = None
          break
        if data['Pc'] is not None:
          data['Pc'] = ((sum * data['╢D1']) + data['Pc_raw']) / 2
      if i == 0:
        data['Pc_psi'] = data['Pc'] / 6.89475729
      else:
        if data['m_ox'] > 0 and data['m_f'] > 0:
          data['Pc_psi'] = CONVERT(data['Pc'] * 1000, "Pa", "psi")
        else:
          data['Pc_psi'] = None
          break
      if i == 0:
        data['ΔP_ox'] = data['P_N2O_inj'] - data['Pc']
      elif i < 10 or i >= 15:
        data['ΔP_ox'] = data['P_N2O_inj'] - data['Pc'] if data['m_ox'] > 0 and data['m_f'] > 0 else None
      elif i < 15:
        data['ΔP_ox'] = ((data['P_N2O_inj'] - data['Pc']) + data['ΔP_ox']) / 2  if data['m_ox'] > 0 and data['m_f'] > 0 else None
      data['ΔP_ox (psi)'] = CONVERT(data['ΔP_ox'] * 1000, "Pa", "psi")
      if i == 0:
        data['ΔP_fu'] = data['P_fu_inj'] - data['Pc']
      elif i < 10 or i >= 15:
        data['ΔP_fu'] = data['P_fu_inj'] - data['Pc'] if data['m_ox'] > 0 and data['m_f'] > 0 else None
      elif i < 15:
        data['ΔP_fu'] = ((data['P_fu_inj'] - data['Pc']) + data['ΔP_fu']) / 2  if data['m_ox'] > 0 and data['m_f'] > 0 else None
      data['ΔP_fu (psi)'] = CONVERT(data['ΔP_fu'] * 1000, "Pa", "psi")
      if i == 1:
        data['Me'] = 1
      elif i > 1:
        match_val = MATCH(data['gam0'], isentropic_flow_data[0], 1)
        if match_val is not None:
            slice_start_col = match_val - 5
            slice_end_col = match_val
            main_vlookup_table_array = slice_data(isentropic_flow_data, 4, 503, slice_start_col, slice_end_col)
            data['Me'] = float(VLOOKUP(expansion_ratio, main_vlookup_table_array, 6, True))
        else:
            data['Me'] = np.nan # Or some other default
      if i == 0:
        data['Pe'] = data['Pc'] / 10
      elif i == 1:
        data['Pe'] = data['Pc'] / 100
      else:
        data['Pe'] = data['Pc'] /(1 + (data['gam0'] - 1) / 2 * data['Me']**2)**(data['gam0'] / (data['gam0'] - 1))
      data['Pe_psi'] = CONVERT(data['Pe'] * 1000, "Pa", "psi")
      if i == 0:
        data['rho_o'] = float(VLOOKUP(data['P_N2O_tank'], slice_data(n2o_data, 3, 231, "B", "N"), 2, True))
      else:
        data['rho_o'] = float(VLOOKUP(data['P_N2O_tank'], slice_data(n2o_data, 3, 231, "B", "N"), 2, True)) if data['N2O Status'] == "Liquid" else float(VLOOKUP(data['P_N2O_tank'], slice_data(n2o_data, 3, 231, "R", "AD"), 2, True))
      if i == 0:
        data['m_dot_o'] = oxidizer_CdA * math.sqrt(2 * data['rho_o'] * (data['ΔP_ox']) * 1000)
      else:
        data['m_dot_o'] = oxidizer_CdA * math.sqrt(2 * data['rho_o'] * (data['ΔP_ox']) * 1000) if (oxidizer_CdA * math.sqrt(2 * data['rho_o'] * (data['ΔP_ox']) * 1000)) < data['m_dot_o'] and data['N2O Status'] == "Gas" else (oxidizer_CdA * math.sqrt(2 * data['rho_o'] * (data['ΔP_ox']) * 1000) + data['m_dot_o']) / 2
      data['Q_o'] = data['m_dot_o'] / data['rho_o']
      if i == 0:
        data['µ_o'] = float(VLOOKUP(data['P_N2O_tank'], slice_data(n2o_data, 3, 231, "B", "N"), 11, True)) / 1000
      else:
        data['µ_o'] = float(VLOOKUP(data['P_N2O_tank'], slice_data(n2o_data, 3, 231, "B", "N"), 11, True)) / 1000 if data['N2O Status'] == "Liquid" else float(VLOOKUP(data['P_N2O_tank'], slice_data(n2o_data, 3, 231, "R", "AD"), 12, True)) / 1000
      data['v_o_line'] = data['m_dot_o'] / (data['rho_o'] * ox_feed_line_area)
      data['Re_o'] = data['rho_o'] * data['v_o_line'] * CONVERT(ox_feed_line_diameter, ox_feed_line_diameter_units, "m") / data['µ_o']
      if i == 0:
        data['cgd_o'] = -2 * math.log10(2.51 / (data['Re_o'] * 0.01**0.5) + (CONVERT(ox_line_roughness, ox_line_roughness_units, "m") / CONVERT(ox_feed_line_diameter, ox_feed_line_diameter_units, "m") / 3.72))
      else:
        data['cgd_o'] = -2 * math.log10(2.51 / (data['Re_o'] * data['dwc_o']**0.5) + (CONVERT(ox_line_roughness, ox_line_roughness_units, "m") / CONVERT(ox_feed_line_diameter, ox_feed_line_diameter_units, "m") / 3.72))
      data['dwc_o'] = 1 / data['cgd_o']**2
      data['Q_o_gpm'] = data['Q_o'] * 15850.3
      data['dP_o_Cv'] = 0 if ox_component_Cv == 0 else CONVERT((data['Q_o_gpm'] / ox_component_Cv)**2 * data['rho_o'] / 1000, "psi", "pa")
      data['dP_o_line'] = data['dwc_o'] * (CONVERT(ox_line_length, ox_line_length_units, "m") / CONVERT(ox_feed_line_diameter, ox_feed_line_diameter_units, "m")) * (data['rho_o'] * data['v_o_line']**2 / 2)
      if i == 0 or i >= 10:
        data['dP_o_total'] = data['dP_o_Cv'] + data['dP_o_line'] + CONVERT(ox_additional_loss, "psi", "pa")
      else:
        data['dP_o_total'] = (data['dP_o_Cv'] + data['dP_o_line'] + CONVERT(ox_additional_loss, "psi", "pa")) * (0.1 * i)
      data['dP_o_total_psi'] = CONVERT(data['dP_o_total'], "Pa", "psi")
      data['rho_f'] = fuel_density
      if i == 0:
        data['m_dot_f'] = fuel_CdA * math.sqrt(2 * data['rho_f'] * (data['P_fu_inj'] - data['Pc']) * 1000)
      else:
        data['m_dot_f'] = fuel_CdA * math.sqrt(2 * data['rho_f'] * data['ΔP_fu'] * 1000) if (fuel_CdA * math.sqrt(2 * data['rho_f'] * data['ΔP_fu'] * 1000)) < data['m_dot_f'] and data['N2O Status'] == "Gas" else (fuel_CdA * math.sqrt(2 * data['rho_f'] * data['ΔP_fu'] * 1000) + data['m_dot_f']) / 2
      data['Q_f'] = data['m_dot_f'] / float(VLOOKUP(fuel, slice_data(fuel_data,4,14,"E","H"), 3, False))
      data['v_f_line'] = data['m_dot_f'] / (float(VLOOKUP(fuel, slice_data(fuel_data,4,14,"E","H"), 3, False)) * f_feed_line_area)
      data['Re_f'] = float(VLOOKUP(fuel, slice_data(fuel_data,4,14,"E","H"), 3, False)) * data['v_f_line'] * CONVERT(f_feed_line_diameter, f_feed_line_diameter_units, "m") / float(VLOOKUP(fuel, slice_data(fuel_data,4,14,"E","H"), 4, False))
      if i == 0:
        data['cgd_f'] = -2 * math.log10(2.51 / (data['Re_f'] * 0.025**0.5) + (CONVERT(f_line_roughness, f_line_roughness_units, "m") / CONVERT(f_feed_line_diameter, f_feed_line_diameter_units, "m") / 3.72))
      else:
        data['cgd_f'] = -2 * math.log10(2.51 / (data['Re_f'] * data['dwc_f']**0.5) + (CONVERT(f_line_roughness, f_line_roughness_units, "m") / CONVERT(f_feed_line_diameter, f_feed_line_diameter_units, "m") / 3.72))
      data['dwc_f'] = 1 / data['cgd_f']**2
      data['Q_f_gpm'] = data['Q_f'] * 15850.3
      data['dP_f_Cv'] = 0 if f_component_Cv == 0 else CONVERT((data['Q_f_gpm'] / f_component_Cv)**2 * data['rho_f'] / 1000, "psi", "pa")
      data['dP_f_line'] = data['dwc_f'] * (CONVERT(f_line_length, f_line_length_units, "m") / CONVERT(f_feed_line_diameter, f_feed_line_diameter_units, "m")) * (data['rho_f'] * data['v_f_line']**2 / 2)
      if i == 0 or i >= 10:
        data['dP_f_total'] = data['dP_f_Cv'] + data['dP_f_line'] + CONVERT(f_additional_loss, "psi", "pa")
      elif i < 10:
        data['dP_f_total'] = (data['dP_f_Cv'] + data['dP_f_line'] + CONVERT(f_additional_loss, "psi", "pa")) * (i * 0.1)
      data['dP_f_total_psi'] = CONVERT(data['dP_f_total'], "Pa", "psi")
      data['m_dot'] = data['m_dot_o'] + data['m_dot_f']
      if i == 0:
        data['m_ox'] = oxidizer_volume * n2o_density / 61024
      else:
        data['m_ox'] = 0 if (data['m_ox'] - data['m_oxgen']) < 0 else data['m_ox'] - data['m_oxgen']
      data['m_oxgen'] = 0 if data['m_ox'] == 0 or data['m_oxgen'] == 0 or data['Status'] == "OD" else data['m_dot_o'] * data['Step']
      if i == 0:
        data['m_fgen'] = data['m_dot_f'] * data['Step']
      else:
        data['m_fgen'] = 0 if data['m_f'] == 0 else data['m_dot_f'] * data['Step']
      if i == 0:
        data['m_f'] = fuel_volume * fuel_density / 61024
      else:
         data['m_f'] = 0 if (data['m_f'] - data['m_fgen']) < 0 else data['m_f'] - data['m_fgen']
      if data['m_dot_o'] == 0 or data['m_dot_f'] == 0:
        if data['MR'] != 0:
          data['MR'] = 0
        else:
          data['MR'] = None
          break
      else:
        data['MR'] = data['m_dot_o'] / data['m_dot_f']
      if i == 1:
        data['T0'] = 1000
      elif i > 1:
        data['T0'] = (c_star_efficiency**2) * get_fuel_property(data['Pc_psi'], c_star_efficiency, data['MR'], t0_table, t0_match_array)
      data['Te'] = data['T0'] / (1 + (data['gam0'] - 1) / 2 * data['Me']**2)
      if i  < 2:
        data['ve'] = data['Me'] * math.sqrt(data['gam0'] * data['R'] * data['Te'])
      else:
        data['ve'] = nozzle_efficiency * data['Me'] * math.sqrt(data['gam0'] * data['R'] * data['Te'])
      if i == 0:
        data['╢D1_1'] = 1
      elif i == 1:
        data['╢D1_1'] = 0.75
      else:
        data['╢D1_1'] = data['F'] / history[1]['F'] if data['F'] < history[1]['F'] else 1
      if i == 0:
        data['╢D2'] = 1
      elif i < 15:
        data['╢D2'] = 1500000 * data['Step']
      else:
        data['╢D2'] = 1
      if i < 2:
        data['F_raw'] = data['m_dot'] * data['ve']
      else:
        if data['F'] < 1 or data['Status'] == "FD" or data['Status'] == "OD":
          data['F_raw'] = 0
        else:
          common_calculation = data['m_dot'] * data['ve'] * (1 + math.cos(math.radians(exit_half_angle))) / 2 + (data['Pe'] * 1000 - CONVERT(ambient_pressure, "psi", "Pa")) * exit_area
          if common_calculation < 0.01:
            data['F_raw'] = 0.01
          else:
            data['F_raw'] = common_calculation
      if i > 0:
         data['dF/dt'] = (data['F_raw'] - data['F']) / data['Step']
      if i == 0:
        data['F'] = data['F_raw']
      elif i < 10:
        data['F'] = (data['╢D2'] * data['Step'] + data['F'] if ((data['F_raw'] - data['F']) / data['Step']) > data['╢D2'] else data['F_raw']) * data['╢D1_1']
      else:
        sum = 0
        if data['m_ox'] > 0 and data['m_f'] > 0:
          if data['N2O Status'] == "Gas" and history[1]['N2O Status'] == "Liquid":
            sum = (data['F_raw'] + data['F']) / 2
          else:
            if SIGN(data['dF/dt']) != SIGN(history[0]['dF/dt']) and SIGN(data['dF/dt']) == SIGN(history[1]['dF/dt']) and SIGN(data['dF/dt']) != SIGN(history[2]['dF/dt']):
              sum = (data['F_raw'] + data['F']) / 2
            else:
              sum = data['F_raw']
        else:
          data['F'] = None
          break
        if data['F'] is not None:
          data['F'] = ((sum * data['╢D1_1']) + data['F']) / 2
      data['F_lbf'] = CONVERT(data['F'], "N", "lbf")
      data['J'] = data['F'] * data['Step']
      if i == 0:
        data['J_deliv'] = 0 if data['Time'] == 0 else data['J']
      else:
        data['J_deliv'] = 0 if data['Time'] == 0 else data['J'] + data['J_deliv']
      if data['Status'] == "FD" or data['Status'] == "OD":
        data['Status'] = data['Status']
      else:
        if data['m_f'] == 0:
          data['Status'] = "FD"
        else:
          if data['m_ox'] == 0:
            data['Status'] = "OD"
          else:
            data['Status'] = "Burning"
      if i < 2:
        data['N2O Status'] = "Liquid"
      else:
        data['N2O Status'] = "Liquid" if data['m_ox'] / init_m_ox > 0.1 else "Gas"
      if i == 0:
        init_P_N2O_tank = data['P_N2O_tank']
        init_m_ox = data['m_ox']
      
      for key, val in data.items():
          if key in results:
              results[key][i] = val
      history.appendleft(data.copy())

    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    pd.set_option('display.colheader_justify', 'left')

    df_calculations = pd.DataFrame(results)
    df_calculations.dropna(subset=['Time'], inplace=True)


    # results
    exit_pressure = calculate_result(df_calculations, "Pe_psi", 7, 14)
    flow_condition = "IDEALLY" if abs(ambient_pressure - exit_pressure) < (ambient_pressure * 0.025) else ("OVER" if exit_pressure < ambient_pressure else "UNDER")
    flow_separation = "NO" if df_calculations.loc[5, "Pe_psi"] > (ambient_pressure / 2) else ("POSSIBLE" if df_calculations.loc[5, "Pe_psi"] > (ambient_pressure * 0.4) else "PROBABLE")
    chamber_pressure = INTERCEPT(df_calculations, "Pc_psi", "Time", 28, 33)
    initial_thrust = INTERCEPT(df_calculations, "F_lbf", "Time", 28, 33)
    burn_time = df_calculations.iloc[-1,0]
    mixture_ratio = INTERCEPT(df_calculations, "MR", "Time", 28, 33)
    mass_flow_rate = INTERCEPT(df_calculations, "m_dot", "Time", 28, 33)
    overall_efficieny = nozzle_efficiency * c_star_efficiency * (1 + math.cos(math.radians(exit_half_angle))) / 2
    tank_ratio = CONVERT(oxidizer_mass, oxidizer_mass_units, "kg") / CONVERT(fuel_mass, fuel_mass_units, "kg")
    oxidizer_injection_delta = CONVERT(df_calculations[df_calculations['ΔP_ox'] > 0]['ΔP_ox'].min() * 1000, "Pa", "psi")
    oxidizer_stiffness = oxidizer_injection_delta / chamber_pressure
    fuel_injection_delta = CONVERT(df_calculations[df_calculations['ΔP_fu'] > 0]['ΔP_fu'].min() * 1000, "Pa", "psi")
    fuel_stiffness = fuel_injection_delta / chamber_pressure
    total_impulse = df_calculations['J'].sum()
    specific_impulse_S = CONVERT(initial_thrust, "lbf", "N") / ((mass_flow_rate) * 9.81)
    specific_impulse_C = total_impulse / ((CONVERT(fuel_mass, fuel_mass_units, "kg") + CONVERT(oxidizer_mass, oxidizer_mass_units, "kg")) * 9.81)

    return {
        "exit_pressure": exit_pressure,
        "flow_condition": flow_condition,
        "flow_separation": flow_separation,
        "chamber_pressure": chamber_pressure,
        "initial_thrust": initial_thrust,
        "burn_time": burn_time,
        "mixture_ratio": mixture_ratio,
        "mass_flow_rate": mass_flow_rate,
        "overall_efficiency": overall_efficieny,
        "tank_ratio": tank_ratio,
        "oxidizer_injection_delta": oxidizer_injection_delta,
        "oxidizer_stiffness": oxidizer_stiffness,
        "fuel_injection_delta": fuel_injection_delta,
        "fuel_stiffness": fuel_stiffness,
        "total_impulse": total_impulse,
        "specific_impulse_S": specific_impulse_S,
        "specific_impulse_C": specific_impulse_C
    }
    
# Define the objective function for the optimizer
def objective_function(params):
    td, ed, cd, cl, eha, oxn, oxd, fn, fd = params

    # Constraints
    if ed <= td or cd <= td:
        return 0  # Return a large number to penalize invalid designs

    try:
        results = run_simulation(
            n2o_data=n2o_data,
            fuel_data=fuel_data,
            formulas_data=formulas_data,
            calculations_data=calculations_data,
            isentropic_flow_data=isentropic_flow_data,
            throat_diameter=td,
            exit_diameter=ed,
            chamber_diameter=cd,
            chamber_length=cl,
            exit_half_angle=eha,
            ox_number_of_holes=int(oxn),
            ox_injector_hole_diameter=oxd,
            f_number_of_holes=int(fn),
            f_injector_hole_diameter=fd
        )
        if results and results.get('total_impulse') is not None:
            # We want to maximize total_impulse
            return results['total_impulse']
        else:
            return 0 # Penalize simulations that fail or produce no impulse
    except Exception:
        return 0 # Penalize simulations that cause errors

def create_individual():
    """Create a random individual within the bounds."""
    individual = []
    for min_val, max_val in bounds:
        if isinstance(min_val, int) and isinstance(max_val, int):
            individual.append(random.randint(min_val, max_val))
        else:
            individual.append(random.uniform(min_val, max_val))
    return individual

def create_initial_population():
    """Create the initial population."""
    return [create_individual() for _ in range(POPULATION_SIZE)]

def crossover(parent1, parent2):
    """Perform crossover between two parents."""
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(individual):
    """Mutate an individual."""
    mutated_individual = []
    for i, (min_val, max_val) in enumerate(bounds):
        if random.random() < MUTATION_RATE:
            if isinstance(min_val, int) and isinstance(max_val, int):
                mutated_individual.append(random.randint(min_val, max_val))
            else:
                mutated_individual.append(random.uniform(min_val, max_val))
        else:
            mutated_individual.append(individual[i])
    return mutated_individual

def selection(population, fitness_scores):
    """Select parents using tournament selection."""
    selected_parents = []
    for _ in range(len(population)):
        tournament = random.sample(list(zip(population, fitness_scores)), TOURNAMENT_SIZE)
        winner = max(tournament, key=lambda x: x[1])
        selected_parents.append(winner[0])
    return selected_parents

## 3. Run Optimization

This cell runs the genetic algorithm to find the optimal engine design.

In [ ]:
# Define the bounds for the optimization
bounds = [
    (0.5, 1.5),    # throat_diameter
    (1.25, 3.25),  # exit_diameter
    (2.0, 4.5),    # chamber_diameter
    (4.0, 6.0),    # chamber_length
    (10, 20),      # exit_half_angle
    (4, 12),       # ox_number_of_holes
    (0.05, 0.4),   # ox_injector_diameter
    (4, 12),       # f_number_of_holes
    (0.05, 0.4)    # f_injector_diameter
]

# Genetic Algorithm Parameters
POPULATION_SIZE = 30
NUM_GENERATIONS = 20
MUTATION_RATE = 0.1
TOURNAMENT_SIZE = 3

print("Starting design optimization with Genetic Algorithm...\n")

# Initialize population
population = create_initial_population()
best_individual = None
best_fitness = -1

for generation in range(NUM_GENERATIONS):
    # Calculate fitness for each individual
    fitness_scores = [objective_function(ind) for ind in population]

    # Find the best individual in the current generation
    current_best_fitness = max(fitness_scores)
    if current_best_fitness > best_fitness:
        best_fitness = current_best_fitness
        best_individual = population[fitness_scores.index(current_best_fitness)]

    print(f"Generation {generation + 1}/{NUM_GENERATIONS} - Best Impulse: {best_fitness:.2f} Ns")

    # Selection
    parents = selection(population, fitness_scores)

    # Crossover and Mutation
    next_generation = []
    for i in range(0, POPULATION_SIZE, 2):
        parent1, parent2 = parents[i], parents[i+1]
        child1, child2 = crossover(parent1, parent2)
        next_generation.append(mutate(child1))
        next_generation.append(mutate(child2))

    population = next_generation

print("\n--- Optimization Complete ---")

## 4. Display Final Results

This cell runs the simulation one last time with the best-found parameters and prints a detailed report.

In [ ]:
if best_individual:
    # Rerun the simulation with the best parameters to get all results
    final_results = run_simulation(
        n2o_data=n2o_data,
        fuel_data=fuel_data,
        formulas_data=formulas_data,
        calculations_data=calculations_data,
        isentropic_flow_data=isentropic_flow_data,
        throat_diameter=best_individual[0],
        exit_diameter=best_individual[1],
        chamber_diameter=best_individual[2],
        chamber_length=best_individual[3],
        exit_half_angle=best_individual[4],
        ox_number_of_holes=int(best_individual[5]),
        ox_injector_hole_diameter=best_individual[6],
        f_number_of_holes=int(best_individual[7]),
        f_injector_hole_diameter=best_individual[8]
    )

    print(f"Optimal Engine Dimensions Found:")
    print(f"  Throat Diameter: {best_individual[0]:.3f} in")
    print(f"  Exit Diameter: {best_individual[1]:.3f} in")
    print(f"  Chamber Diameter: {best_individual[2]:.3f} in")
    print(f"  Chamber Length: {best_individual[3]:.2f} in")
    print(f"  Exit Half Angle: {best_individual[4]:.1f}°")
    print(f"  Ox Injector Holes: {int(best_individual[5])}")
    print(f"  Ox Injector Diameter: {best_individual[6]:.4f} in")
    print(f"  Fuel Injector Holes: {int(best_individual[7])}")
    print(f"  Fuel Injector Diameter: {best_individual[8]:.4f} in")
    print(f"  Maximum Total Impulse: {best_fitness:.2f} Ns")
    
    print("\n--- Full Simulation Results for Optimal Design ---")
    for key, value in final_results.items():
        if isinstance(value, float):
            print(f"  {key.replace('_', ' ').title()}: {value:.4f}")
        else:
            print(f"  {key.replace('_', ' ').title()}: {value}")
else:
    print("Optimization did not find a solution.")